If you're opening this Notebook on colab, you will probably need to install the most recent versions of 🤗 Transformers and 🤗 Datasets. We will also need `scipy` and `scikit-learn` for some of the metrics. Uncomment the following cell and run it.

In [1]:
! pip install transformers
! pip install datasets
! pip install scipy sklearn
! pip install huggingface_hub

Check if tensorflow and GPUs are configured correctly

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

2022-12-24 12:52:27.308303: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-24 12:52:27.548601: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  2


If you're opening this notebook locally, make sure your environment has an install from the latest version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow:

First you have to create an access token on the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then uncomment the following cell and input your token.

In [3]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /home/ali/.huggingface/token


Then you need to install Git-LFS and setup Git if you haven't already. Uncomment the following instructions and adapt with your name and email:

In [4]:
# !apt install git-lfs
!git config --global user.email "idfassahata@gmail.com"
!git config --global user.name "fassahat"

Make sure your version of Transformers is at least 4.16.0 since the functionality was introduced in that version:

In [5]:
import transformers

print(transformers.__version__)

4.23.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/text-classification).

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).

![Widget inference on a text classification task](https://github.com/huggingface/notebooks/blob/main/examples/images/text_classification.png?raw=1)

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use Keras to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (a task with the same training set as `mnli` but different validation and test sets):

In [6]:
GLUE_TASKS = [
    "cola",
    "mnli",
    "mnli-mm",
    "mrpc",
    "qnli",
    "qqp",
    "rte",
    "sst2",
    "stsb",
    "wnli",
]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on your model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set these three parameters, then the rest of the notebook should run smoothly:

In [7]:
task = "sst2"
model_checkpoint = "distilbert-base-uncased"
model_cp = "bert-for-patents"
batch_size = 8

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and the [🤗 Evaluate](https://github.com/huggingface/datasets) library to get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the `load_dataset` function from `datasets` and and the `load` function from `evaluate`. 

In [8]:
from datasets import load_dataset
! pip install evaluate
from evaluate import load

With the exception of `mnli-mm`, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

Connection for Google Drive so one can import files easily while working on Google COLAB

In [9]:
# from google.colab import drive 
# drive.mount('/content/gdrive')

In [9]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load("glue", actual_task)

from datasets import load_dataset

# loading patent sentence dataset
paragraph_data = load_dataset("csv", data_files="/home/ali/notebooks/paragraph_model_predicted_sentences.csv")

Found cached dataset glue (/home/ali/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Using custom data configuration default-859052c41b45c852


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

/home/ali/anaconda3/envs/new_env/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/ali/.cache/huggingface/datasets/csv/default-859052c41b45c852/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Randomizing and Divinding patent sentence data to train and test/validation parts

In [10]:
paragraph_data_randomized = paragraph_data["train"].shuffle(seed=42)
paragraph_data_randomized_rows = paragraph_data.num_rows['train']
train_end = round(paragraph_data_randomized_rows * 0.80) #take atleast 80% samples for training

paragraph_train_dataset = paragraph_data_randomized.select([i for i in list(range(train_end))])
paragraph_test_dataset = paragraph_data_randomized.select([i for i in list(range(train_end, paragraph_data_randomized_rows))])

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(paragraph_train_dataset)

,sentence,label
0,"According to the present invention, a grain-oriented electrical steel sheet having a very low core loss can be manufactured in an expensive and efficient manner by minimizing the contents of Al, which forms Al-based oxides precipitates, N, and Mn, which form MnS precipitates, in a steel making step, while causing secondary recrystallization using grain boundary-segregated elementary S and FeS precipitates as grain boundary inhibitors required for formation of secondary recrystallized grains, and reducing the amount of Al-based precipitates and oxides in the secondarily recrystallized steel sheet to minimize interference with the movement of magnetic domains.",1
1,"', 'Further, it is possible to obtain a display device such as a mobile device, a touch panel or a flat screen television, using such glass for chemical tempering as a glass plate for a display device, such as a cover glass.",1
2,"', 'According to the present invention, there is provided an information processing method, including: obtaining first execution information which indicates an execution state of a first execution unit configured to execute an application program; storing the first execution information; and executing the application program by using the stored first execution information.",0
3,"In order to attain the foregoing object, the inventors of the present invention diligently studied a method for forming a particle film uniformly even on a substrate of a practical size.",0
4,"', 'Therefore, a desirable electromagnetic radiation can be generated efficiently.",1
5,"', 'In view of the foregoing, the inventors have further improved the aluminum frame material disclosed in Patent Literature 2, and have found that the glittering under irradiation with collected light due to the specific crystallized products remaining in the film, such as the Mg2Si crystallized product, can be reduced even when the anodic oxide film is formed by using the alkaline electrolytic solution containing as an electrolyte an organic acid salt as described above by allowing an Al—Zn—Mg-based aluminum alloy for forming the aluminum frame material to satisfy the JIS A7075 standard and have a component composition that has the contents of Mg, Cu, Cr, Fe, and Si restricted.",0
6,"', 'Aspects below are preferred as embodiments of the present invention.",0
7,a method of switching the uplink and the downlink simultaneously to another wireless communication network of the same type.,2
8,Progresses in Medicine and electronic devices have brought about rapid progress in current medical locations.,2
9,An object of the present invention devised to solve the problem lies in a method for controlling transmission power of a sounding reference signal in a wireless communication system and an apparatus for the same.,2


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [14]:
metric

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=ref

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [15]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5625}

Note that `load` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [16]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("anferico/bert-for-patents")

You can directly call this tokenizer on one sentence or a pair of sentences:

In [17]:
tokenizer("Hello, this is a sentence!", "And this sentence goes with it.")

{'input_ids': [2, 7257, 1015, 1688, 1668, 1042, 5916, 1004, 3, 1663, 1688, 5916, 3297, 1672, 1674, 1017, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [18]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [19]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {paragraph_train_dataset[0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: The method includes receiving, from a base station, scheduling information on a control channel of a first cell, and receiving, from the base station, data on a shared channel of a second cell based on the scheduling information.


We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the arguments `truncation=True` and `padding='longest`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model, and all inputs will be padded to the maximum input length to give us a single input array. A more performant method that reduces the number of padding tokens is to write a generator or `tf.data.Dataset` to only pad each *batch* to the maximum length in that batch, but most GLUE tasks are relatively quick on modern GPUs either way.

In [20]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, max_length=128, add_special_tokens=True, padding=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, max_length=128, add_special_tokens=True, padding=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [21]:
preprocess_function(paragraph_train_dataset[:5])

{'input_ids': [[2, 1661, 3783, 2615, 4574, 1015, 1678, 1042, 2583, 1941, 1015, 19605, 2257, 1671, 1042, 2156, 2814, 1662, 1042, 1699, 3191, 1015, 1663, 4574, 1015, 1678, 1661, 2583, 1941, 1015, 2616, 1671, 1042, 3872, 2814, 1662, 1042, 1782, 3191, 1906, 1671, 1661, 19605, 2257, 1017, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1010, 1015, 1010, 1674, 1668, 32134, 1673, 1661, 6587, 1011, 1043, 1012, 1703, 1042, 5192, 20939, 30182, 1971, 1042, 2511, 1662, 1020, 1017, 1019, 25971, 10455, 2348, 1665, 1023, 1017, 1020, 25971, 10455, 2348, 6308, 1017, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [22]:
pre_tokenizer_columns = set(paragraph_train_dataset.features)
encoded_dataset_train = paragraph_train_dataset.map(preprocess_function, batched=True)
encoded_dataset_test = paragraph_test_dataset.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset_train.features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

Columns added by tokenizer: ['input_ids', 'token_type_ids', 'attention_mask']


In [23]:
encoded_dataset_train.features["label"]

Value(dtype='int64', id=None)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `TFAutoModelForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which is always 2, except for STS-B which is a regression problem and MNLI where we have 3 labels).

In [24]:
from transformers import TFAutoModelForSequenceClassification

num_labels = 3 if task.startswith("mnli") else 1 if task == "stsb" else 2
if task == "stsb":
    num_labels = 1
elif task.startswith("mnli"):
    num_labels = 3
else:
    num_labels = 2
    
# This next little bit is optional, but will give us cleaner label outputs later
# If you're using a task other than CoLA, you will probably need to change these
# to match the label names for your task!
id2label = {0: "0", 1: "1", 2: "2"}
label2id = {val: key for key, val in id2label.items()}

# model = TFAutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
# )

model = TFAutoModelForSequenceClassification.from_pretrained(
    "anferico/bert-for-patents", num_labels=3, id2label=id2label, label2id=label2id
)

2022-12-24 12:55:52.841486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-24 12:55:54.176716: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22307 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:3b:00.0, compute capability: 8.0
2022-12-24 12:55:54.178462: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22307 MB memory:  -> device: 1, name: NVIDIA A30, pci bus id: 0000:d8:00.0, compute capability: 8.0
2022-12-24 12:55:56.118989: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the mat

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

Next, we convert our datasets to `tf.data.Dataset`, which Keras understands natively. There are two ways to do this - we can use the slightly more low-level [`Dataset.to_tf_dataset()`](https://huggingface.co/docs/datasets/package_reference/main_classes#datasets.Dataset.to_tf_dataset) method, or we can use [`Model.prepare_tf_dataset()`](https://huggingface.co/docs/transformers/main_classes/model#transformers.TFPreTrainedModel.prepare_tf_dataset). The main difference between these two is that the `Model` method can inspect the model to determine which column names it can use as input, which means you don't need to specify them yourself. Unless our samples are all the same length, we will also need to pass a `tokenizer` or `collate_fn` so that the `tf.data.Dataset` knows how to pad and combine samples into a batch.

In [25]:
validation_key = (
    "validation_mismatched"
    if task == "mnli-mm"
    else "validation_matched"
    if task == "mnli"
    else "validation"
)

tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset_train,
    shuffle=True,
    batch_size=8,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset_test,
    shuffle=False,
    batch_size=8,
    tokenizer=tokenizer,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Next, we need to set up our optimizer and `compile()` our model. The `create_optimizer` function in the Transformers library creates a very useful `AdamW` optimizer with weight and learning rate decay. This performs very well for training most transformer networks - we recommend using it as your default unless you have a good reason not to! Note, however, that because it decays the learning rate over the course of training, it needs to know how many batches it will see during training.

Note that all models in `transformers` can pick a sensible loss function by default. To use this loss, simply do not pass a `loss` argument to `compile()`. Although the losses for GLUE tasks are usually just simple cross-entropy, this can be very helpful in models when the loss is intricate and contains multiple terms.

In some of our other examples, we use `jit_compile` to compile the model with [XLA](https://www.tensorflow.org/xla). In this case, we should be careful about that - because our inputs have variable sequence lengths, we may end up having to do a new XLA compilation for each possible length, because XLA compilation expects a static input shape! For small datasets, this will probably result in spending more time on XLA compilation than actually training, which isn't very helpful.

If you really want to use XLA without these problems (for example, if you're training on TPU), you can create a tokenizer with `padding="max_length"`. This will pad all of your samples to the same length, ensuring that a single XLA compilation will suffice for your entire dataset. Note that depending on the nature of your dataset, this may result in a lot of wasted computation on padding tokens!

In [26]:
from transformers import create_optimizer

num_epochs = 3
batches_per_epoch = len(encoded_dataset_train) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


The last thing to define is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier. The only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B).

In addition, let's wrap this metric computation function in a `KerasMetricCallback`. This callback will compute the metric on the validation set each epoch, including printing it and logging it for other callbacks like `TensorBoard` and `EarlyStopping`.

Why do it this way, though, and not just use a straightforward Keras Metric object? This is a good question - on this task, several of the metrics such as Accuracy are very straightforward, and it would probably make more sense to just use a Keras metric for those instead. However, we want to demonstrate the use of `KerasMetricCallback` here, because it can handle any arbitrary Python function for the metric computation. This turns out to be very important for other NLP tasks like summarization and translation, where standard metrics like `BLEU` and `ROUGE` are much more complex to compute, and often involve decoding tokens generated by the model to strings and comparing their similarity to target sentences. If you want to stop training once `ROUGE` scores on the validation set start to decline, then `KerasMetricCallback` is essential.

That said, if you're only interested in tasks like text classification with straightforward metrics, then by all means remove the `KerasMetricCallback` and use a Keras `Accuracy` metric instead!

With that out of the way, how do we actually use `KerasMetricCallback`? It's straightfoward: We simply define a function that computes metrics given a tuple of numpy arrays of predictions and labels, then we pass that, along with the validation set to compute metrics on, to the callback:

In [27]:
from transformers.keras_callbacks import KerasMetricCallback


def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

We can now finetune our model by just calling the `fit` method. Be sure to pass the TF datasets, and not the original datasets! We can also add a callback to sync up our model with the Hub - this allows us to resume training from other machines and even test the model's inference quality midway through training! Make sure to change the `username` if you do. If you don't want to do this, simply remove the callbacks argument in the call to `fit()`.

In [28]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

push_to_hub_model_id = "anferico-bert-for-patents-finetuned-150k-patent-optimized-sentences"

tensorboard_callback = TensorBoard(log_dir="./anferico_bert_for_patents_150k_patent_optimized_sentences/logs")
push_to_hub_callback = PushToHubCallback(
    output_dir="./anferico_bert_for_patents_150k_patent_optimized_sentences",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
    save_strategy="epoch"
)

callbacks = [metric_callback, tensorboard_callback]

model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=callbacks
)

/home/ali/anaconda3/envs/new_env/lib/python3.10/site-packages/huggingface_hub/repository.py:705: FutureWarning: Creating a repository through 'clone_from' is deprecated and will be removed in v0.11.
  warnings.warn(
Cloning https://huggingface.co/fassahat/anferico-bert-for-patents-finetuned-150k-patent-optimized-sentences into local empty directory.


Epoch 1/3
15000/15000 [==============================] - 2482s 164ms/step - loss: 0.1707 - val_loss: 0.1101 - accuracy: 0.9582
Epoch 2/3
15000/15000 [==============================] - 2467s 164ms/step - loss: 0.0638 - val_loss: 0.1107 - accuracy: 0.9647
Epoch 3/3
15000/15000 [==============================] - 2467s 164ms/step - loss: 0.0149 - val_loss: 0.1131 - accuracy: 0.9711


Saving the fintuned model

In [29]:
import os
os.makedirs("/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/model")
model.save_pretrained("/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/model")
os.makedirs("/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/tokenizer")
tokenizer.save_pretrained("/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/tokenizer")

('/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/tokenizer/tokenizer_config.json',
 '/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/tokenizer/special_tokens_map.json',
 '/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/tokenizer/vocab.txt',
 '/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/tokenizer/added_tokens.json',
 '/home/ali/notebooks/models/anferico_bert_for_patents_150k_optimized_sentences/tokenizer/tokenizer.json')

In [31]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# You can, of course, use your own username and model name here once you've pushed your model using the code above!
model = TFAutoModelForSequenceClassification.from_pretrained("/home/ali/notebooks/models/distilbert_base_uncased_557k/model")
tokenizer = AutoTokenizer.from_pretrained("/home/ali/notebooks/models/distilbert_base_uncased_557k/tokenizer")

Some layers from the model checkpoint at /home/ali/notebooks/models/distilbert_base_uncased_557k/model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /home/ali/notebooks/models/distilbert_base_uncased_557k/model and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

In [32]:
sentences = [
    "The judge told the jurors to think carefully.",
    "The judge told that the jurors to think carefully."
]

In [34]:
tokenized = tokenizer(sentences, return_tensors="np", padding="longest")

outputs = model(tokenized).logits

classifications = np.argmax(outputs, axis=1)
print(classifications)

[0 0]


In [35]:
classifications = [model.config.id2label[output] for output in classifications]
print(classifications)

['0', '0']
